# Diseño de Prompts y Ejecución
1. Instalación `transformers`.
2. 

In [1]:
%pip install transformers
%pip install pandas
%pip install tensorflow
%pip install torch
%pip install tf-keras

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip in

In [2]:
from transformers import pipeline
import pandas as pd

/Users/olivia/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/olivia/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = pd.read_csv("input/yelp_review.csv")
reviews = df["texto"].tolist()

In [4]:
models = {
    "roberta": pipeline("sentiment-analysis", model="siebert/sentiment-roberta-large-english"),
    "bert": pipeline("text-classification", model="nlptown/bert-base-multilingual-uncased-sentiment"),
    "flan-t5": pipeline("text2text-generation", model="google/flan-t5-base")
}

Device set to use mps:0
Device set to use mps:0
Device set to use mps:0


In [ ]:
def zero_shot_prompt(text):
    return (
        "Classify the sentiment of this review as positive, negative or neutral.\n"
        f"Review: {text}\n"
        "Sentiment:"
    )

def few_shot_prompt(text):
    return (
        "Analyze the review and decide if it's positive, negative or neutral, justifying your reasoning.\n\n"
        
        "Review: I’ve been using this phone for over a month now and I honestly couldn’t be happier. "
        "The performance is super fast — apps open instantly, and multitasking is smooth. The camera quality is outstanding, "
        "especially in low light conditions, and the battery easily lasts me a full day with heavy use. It also feels sturdy and well-built. "
        "Totally worth the price, highly recommended.\n"
        "Sentiment: positive\n\n"
        
        "Review: I had high hopes for this product, but I was extremely disappointed. "
        "After just one week of normal use, it completely stopped working. The screen froze constantly, and the battery would not hold a charge. "
        "I tried contacting customer service, but they never responded. Very poor quality for the price, and I regret buying it. "
        "Would not recommend this to anyone.\n"
        "Sentiment: negative\n\n"

        f"Review: {text}\n"
        "Sentiment:"
    )

def cot_prompt(text):
    return (
        "Analyze the review and decide if it's positive, negative or neutral, justifying your reasoning.\n\n"

        "Review: I’ve been using this phone for over a month now and I honestly couldn’t be happier. "
        "The performance is super fast — apps open instantly, and multitasking is smooth. The camera quality is outstanding, "
        "especially in low light conditions, and the battery easily lasts me a full day with heavy use. It also feels sturdy and well-built. "
        "Totally worth the price, highly recommended.\n"
        "Reasoning: The reviewer is clearly satisfied, using words like 'super fast', 'outstanding', and 'highly recommended'. "
        "These indicate a strong positive experience.\n"
        "Sentiment: positive\n\n"

        "Review: I had high hopes for this product, but I was extremely disappointed. "
        "After just one week of normal use, it completely stopped working. The screen froze constantly, and the battery would not hold a charge. "
        "I tried contacting customer service, but they never responded. Very poor quality for the price, and I regret buying it.\n"
        "Reasoning: The reviewer describes multiple failures and dissatisfaction, using phrases like 'extremely disappointed', 'poor quality', and 'regret buying it'.\n"
        "Sentiment: negative\n\n"

        f"Review: {text}\n"
        "Reasoning:"
    )


In [7]:
for name, model in models.items():
    print(f"\n=== {name.upper()} ===\n")
    for text in reviews[:10]:
        prompt = zero_shot_prompt(text)
        if name == "flan-t5":
            result = model(prompt, max_length=50)[0]['generated_text']
        else:
            result = model(prompt)[0]['label']
        print(f"Review: {text}\nPrediction: {result}\n")


=== ROBERTA ===

Review: dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank.
Prediction: POSITIVE

Review: Unfortunately, the frustration of being Dr. Goldberg's patient is a repeat of the experience I've had with so many other doctors in NYC -- good doctor, terrible staff.  It seems that his staff simply never answers the phone.  It usually takes 2 hours of repeated calling to get an answer.  Who has time for that or wants to deal with it?  I have run into this problem with many other doctors and I just don't 

In [12]:
for name, model in models.items():
    print(f"\n=== {name.upper()} ===\n")
    for text in reviews[:10]:
        prompt = few_shot_prompt(text)
        if name == "flan-t5":
            result = model(prompt, max_length=50)[0]['generated_text']
        else:
            result = model(prompt)[0]['label']
        print(f"Review: {text}\nPrediction: {result}\n")


=== ROBERTA ===

Review: dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank.
Prediction: NEGATIVE

Review: Unfortunately, the frustration of being Dr. Goldberg's patient is a repeat of the experience I've had with so many other doctors in NYC -- good doctor, terrible staff.  It seems that his staff simply never answers the phone.  It usually takes 2 hours of repeated calling to get an answer.  Who has time for that or wants to deal with it?  I have run into this problem with many other doctors and I just don't 

In [13]:
for name, model in models.items():
    print(f"\n=== {name.upper()} ===\n")
    for text in reviews[:10]:
        prompt = cot_prompt(text)
        if name == "flan-t5":
            result = model(prompt, max_length=50)[0]['generated_text']
        else:
            result = model(prompt)[0]['label']
        print(f"Review: {text}\nPrediction: {result}\n")


=== ROBERTA ===

Review: dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank.
Prediction: NEGATIVE

Review: Unfortunately, the frustration of being Dr. Goldberg's patient is a repeat of the experience I've had with so many other doctors in NYC -- good doctor, terrible staff.  It seems that his staff simply never answers the phone.  It usually takes 2 hours of repeated calling to get an answer.  Who has time for that or wants to deal with it?  I have run into this problem with many other doctors and I just don't 

Token indices sequence length is longer than the specified maximum sequence length for this model (538 > 512). Running this sequence through the model will result in indexing errors


Review: Got a letter in the mail last week that said Dr. Goldberg is moving to Arizona to take a new position there in June.  He will be missed very much.  \n\nI think finding a new doctor in NYC that you actually like might almost be as awful as trying to find a date!
Prediction: 1 star



RuntimeError: The size of tensor a (538) must match the size of tensor b (512) at non-singleton dimension 1